In [638]:
import pandas as pd

participant_number = 6170

## Define Functions

In [639]:
def process_evaluation_type_sheet(participant_number, sheet_name):
    try:
        df = pd.read_excel(f"{participant_number}.xlsx", sheet_name=sheet_name, header=None)
        df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)  # Remove whitespace
        df.columns = ["WORD", "IS_CORRECT"]  # Assign column names
        for i in range(len(df)):
            if df["IS_CORRECT"][i] == "True":
                df["IS_CORRECT"][i] = True
            else:
                df["IS_CORRECT"][i] = False
        print(f"{sheet_name} sheet processed successfully.")
        return df
    except Exception as e:
        return None

def process_training_type_sheet(participant_number, sheet_name):
    try:
        df = pd.read_excel(f"{participant_number}.xlsx", sheet_name=sheet_name, header=None)
        df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)  # Remove whitespace
        df.columns = ["WORD", "IS_CORRECT", "RESPONSE_TIME"]  # Assign column names
        for i in range(len(df)):
            if df["IS_CORRECT"][i] == "True":
                df["IS_CORRECT"][i] = True
            else:
                df["IS_CORRECT"][i] = False
        df["RESPONSE_TIME"] = df["RESPONSE_TIME"].astype(float)  # Convert "RESPONSE_TIME" to float
        print(f"{sheet_name} sheet processed successfully.")
        return df
    except Exception as e:
        return None

## Load Sheets

In [640]:
columns_for_evaluation_type = ["WORD", "IS_CORRECT"]
df_pre = process_evaluation_type_sheet(participant_number, "PRE")
df_mid = process_evaluation_type_sheet(participant_number, "MID")
df_post = process_evaluation_type_sheet(participant_number, "POST")

PRE sheet processed successfully.


In [641]:
columns_for_training_type = ["WORD", "IS_CORRECT", "RESPONSE_TIME"]
df_session1 = process_training_type_sheet(participant_number, "Session 1")
df_session2 = process_training_type_sheet(participant_number, "Session 2")
df_session3 = process_training_type_sheet(participant_number, "Session 3")
df_session4 = process_training_type_sheet(participant_number, "Session 4")
df_session5 = process_training_type_sheet(participant_number, "Session 5")
df_session6 = process_training_type_sheet(participant_number, "Session 6")
df_session7 = process_training_type_sheet(participant_number, "Session 7")
df_session8 = process_training_type_sheet(participant_number, "Session 8")

Session 1 sheet processed successfully.


/var/folders/60/ssxfrvx144d5wrq4gqq06rfw0000gp/T/ipykernel_16130/3113563077.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["IS_CORRECT"][i] = True
/var/folders/60/ssxfrvx144d5wrq4gqq06rfw0000gp/T/ipykernel_16130/3113563077.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["IS_CORRECT"][i] = True
/var/folders/60/ssxfrvx144d5wrq4gqq06rfw0000gp/T/ipykernel_16130/3113563077.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [642]:
df_mini1 = process_evaluation_type_sheet(participant_number, "Mini 1")
df_mini2 = process_evaluation_type_sheet(participant_number, "Mini 2")
df_mini3 = process_evaluation_type_sheet(participant_number, "Mini 3")
df_mini4 = process_evaluation_type_sheet(participant_number, "Mini 4")
df_mini5 = process_evaluation_type_sheet(participant_number, "Mini 5")
df_mini6 = process_evaluation_type_sheet(participant_number, "Mini 6")
df_mini7 = process_evaluation_type_sheet(participant_number, "Mini 7")
df_mini8 = process_evaluation_type_sheet(participant_number, "Mini 8")

Mini 1 sheet processed successfully.


## Compare PRE, MID, POST

In [643]:
# compare accuracy of PRE, MID, POST
def get_accuracy_average(df):    # cast to bool
    if df is None:
        return None
    else:
        return df["IS_CORRECT"].value_counts().get(True, 0) / len(df) * 100

In [644]:
pre_accuracy = get_accuracy_average(df_pre)
mid_accuracy = get_accuracy_average(df_mid)
post_accuracy = get_accuracy_average(df_post)

result = {
    "PRE": pre_accuracy,
    "MID": mid_accuracy,
    "POST": post_accuracy
}

print(result)

{'PRE': 87.5, 'MID': None, 'POST': None}


## Analyze Between Sessions

In [645]:
df_sessions = [df_session1, df_session2, df_session3, df_session4, df_session5, df_session6, df_session7, df_session8]
df_sessions = [df for df in df_sessions if df is not None]


In [646]:
def analyze_pre():
    if df_pre is None:
        return None

    # count the number of "True" in the "IS_CORRECT" column
    df = pd.DataFrame(df_pre)
    # cast to bool
    print(df_pre["IS_CORRECT"][0] == True)

analyze_pre()



True
